In [25]:
import os
import time
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torch
from PIL import Image
from sklearn.metrics import accuracy_score

In [26]:
path_to_model = "weights/my_model"

extractor = AutoFeatureExtractor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model)

/home/kirill/.local/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [27]:
def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits

    predicted_label = logits.argmax(-1).item()

    return model.config.id2label[predicted_label]

In [28]:
path = "data/"
images_list = os.listdir(path)

In [29]:
def size_measurement(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / (1024 ** 2)
    print('model size: {:.3f}MB'.format(size_all_mb))

In [30]:
# Найдем исходный размер модели.
size_measurement(vit_model)

model size: 327.302MB


In [32]:
import torch.nn.utils.prune as prune
import copy

# Multimodal structured pruning

In [36]:
model_prune = copy.deepcopy(vit_model)

for name, module in model_prune.named_modules():
    if isinstance(module, torch.nn.Linear):
        module = prune.ln_structured(module, name='weight', amount=0.05, n='fro', dim=-1)
        module = prune.random_structured(module, name='weight', amount=0.01, dim=-1)
        prune.remove(module, name='weight')
        

In [39]:
start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in images_list:

    image = Image.open(path + element, mode='r', formats=None)

    inputs = extractor(image, return_tensors="pt")
    predict = model_use(model_prune, inputs)
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

acc = accuracy_score(target_list, predict_list)
print("Точность модели после прунинга= ", acc)
print("Время обработки изображений модели после прунинга= ", end_time-start_time, " секунд")
print("Скорость обработки изображений у модели после прунинга составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")

Точность модели после прунинга=  0.95625
Время обработки изображений модели после прунинга=  19.01687526702881  секунд
Скорость обработки изображений у модели после прунинга составила   8.413579925899064  картинок в секунду


In [40]:
# Найдем исходный размер модели.
size_measurement(model_prune)

model size: 327.302MB
